# Part 2: Training a surrogate model with `rholearn` (supporting notebook)

In [ ]:
import rholearn

from dft_settings import DFT_SETTINGS
from ml_settings import ML_SETTINGS
from net import NET

rholearn.train(DFT_SETTINGS, ML_SETTINGS, NET)

In [ ]:
import rholearn

import dft_settings
import ml_settings
import hpc_settings

rholearn.eval(
    dft_settings.DFT_SETTINGS, ml_settings.ML_SETTINGS, hpc_settings.HPC_SETTINGS
)

In [ ]:
import torch

from rholearn.model import RhoModel
from rholearn.utils import system


model = RhoModel(
    target_basis=TARGET_BASIS,
    spherical_expansion_hypers=SPHERICAL_EXPANSION_HYPERS,
    net=NET,
)
model

In [8]:
# Dataset
from rholearn import train_utils

dataset = train_utils.get_dataset(
    FRAMES, FRAME_IDXS, model, TRAIN_DATA_NAMES, PROCESSED_DIR, OVERLAP_CUTOFF, **TORCH,
)

In [9]:
dataloader = train_utils.get_dataloader(
    dataset,
    dloader_kwargs={"batch_size": 10, "shuffle": True},
)

In [10]:
batch = next(iter(dataloader))
batch.target_c

In [ ]:
batch.sample_id

In [ ]:
from rholearn.utils import convert

input_c = model(  # forward pass
    frames=batch.frame,
    frame_idxs=batch.sample_id,
    descriptor=batch.descriptor,
    split_by_frame=False,
    check_metadata=True,
)
input_c = convert.coeff_vector_to_sparse_by_center_type(input_c, backend="torch")
input_c[0].samples

In [ ]:
batch.descriptor[0].samples

In [ ]:
import metatensor.torch as mts
from rholearn.loss import RhoLoss
from rholearn.utils import convert

RhoLoss(
    # normalized=False,
    orthogonal=False,
    # truncated=True,
)(
    input_c=input_c,
    target_c=batch.target_c,
    target_w=batch.target_w,
    overlap=batch.overlap,
    check_metadata=False,
)

In [ ]:
import metatensor.torch as mts

batch.target_c[0].samples.values[:10], input_c[0].samples.values[:10]

In [ ]:
input_c[0].samples

In [ ]:
from rholearn.utils import convert

convert.coeff_vector_to_sparse_by_center_type(input_c, backend="torch")

In [ ]:
len(batch.overlap[0].samples)

In [ ]:
import numpy as np

train_utils.crossval_idx_split(np.arange(100), 50, 40, 10)